# Day 18 
## Part 1

I really need a library of utilities rather than copying and pasting this every time.

Fill the trench in the same way as in day 10, i.e. do a search and one that doesn't hit a boundary contains the inner points.

I've a horrible feeling Part 2 will consist of doing this for an enormous number of steps, not sure how I'll deal with that.

In [1]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __hash__(self):
        return hash((self.x, self.y))

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    

N = Point(0, 1)
S = Point(0, -1)
W = Point(-1, 0)
E = Point(1, 0)


def manhattan_distance(p1, p2):
    return abs(p1.x - p2.x) + abs(p1.y - p2.y)

In [2]:
import parse

def parse_data(input):
    instructions = []
    for line in input.strip().splitlines():
        r = parse.parse("{direction} {steps:d} ({colour:x})", line.replace('#', '0x'))
        instructions.append(r.named)
    return instructions

test_data = parse_data("""R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)""")

test_data

[{'direction': 'R', 'steps': 6, 'colour': 7390992},
 {'direction': 'D', 'steps': 5, 'colour': 902513},
 {'direction': 'L', 'steps': 2, 'colour': 5706736},
 {'direction': 'D', 'steps': 2, 'colour': 13811841},
 {'direction': 'R', 'steps': 2, 'colour': 5883520},
 {'direction': 'D', 'steps': 2, 'colour': 4266897},
 {'direction': 'L', 'steps': 5, 'colour': 9236194},
 {'direction': 'U', 'steps': 2, 'colour': 13279603},
 {'direction': 'L', 'steps': 1, 'colour': 1792162},
 {'direction': 'U', 'steps': 2, 'colour': 13279601},
 {'direction': 'R', 'steps': 2, 'colour': 7866322},
 {'direction': 'U', 'steps': 3, 'colour': 10977187},
 {'direction': 'L', 'steps': 2, 'colour': 86578},
 {'direction': 'U', 'steps': 2, 'colour': 8004067}]

In [3]:
directions = {
    "R": E,
    "L": W,
    "U": N,
    "D": S
}

def dig_trench(instructions):
    trench = {}
    p = Point(0, 0)
    for i in instructions:
        for _ in range(i["steps"]):
            p += directions[i["direction"]]
            trench[p] = i["colour"]
    return trench

test_trench = dig_trench(test_data)
assert len(test_trench) == 38

def print_points(ps):
    lines = []
    min_x = min(p.x for p in ps)
    max_x = max(p.x for p in ps)
    min_y = min(p.y for p in ps)
    max_y = max(p.y for p in ps)
    for y in range(max_y, min_y - 1, -1):
        lines.append(
            "".join(
                "#" if Point(x, y) in ps else "."
                for x in range(min_x, max_x + 1)
            )
        )
    print("\n".join(lines))

In [4]:
from copy import deepcopy

def fill_trench(trench):
    ps = deepcopy(trench)
    min_x = min(p.x for p in ps)
    max_x = max(p.x for p in ps)
    min_y = min(p.y for p in ps)
    max_y = max(p.y for p in ps)

    inner = set()
    outer = set()
    
    n = 0
    
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            p = Point(x, y)
            if p not in ps and p not in inner and p not in outer:
                q = [p]
                connected = {p}
                hit_boundary = False
                while q:
                    loc = q.pop()
                    for d in (N, S, E, W):
                        nbr = loc + d
                        if nbr not in ps and nbr not in connected:
                            if min_x <= nbr.x <= max_x and min_y <= nbr.y <= max_y:
                                q.append(nbr)
                                connected.add(nbr)
                                n += 1
                            else:
                                hit_boundary = True
                if not hit_boundary:
                    inner.update(connected)
                else:
                    outer.update(connected)

    for i in inner:
        ps[i] = 0

    return(ps)

def part_1(data):
    return len(fill_trench(dig_trench(data)))

assert part_1(test_data) == 62

In [5]:
data = parse_data(open("input").read())

part_1(data)

74074

In [6]:
%%time

part_1(data)

CPU times: user 1.06 s, sys: 16.7 ms, total: 1.08 s
Wall time: 1.08 s


74074

## Part 2

Of course. Will have to track the coordinates of the corners somehow and do something with overlaps, at a guess.